In [1]:
##Preable
from __future__ import division
import numpy as np
import pandas as pd
from matplotlib.patches import Polygon
import matplotlib.pyplot as plt
from matplotlib import gridspec
import copy
import pickle
import emcee
import corner

from EightBitTransit.cTransitingImage import TransitingImage
from EightBitTransit.inversion import *
from EightBitTransit.misc import *

%matplotlib inline

In [2]:
def fillShape(grid, shapes):
    """Creates a grid with the shape bound by given points filled in.
    Points in the sublists are connected in the order they are given
    Sublists are not connected to each other

    Args:
        grid (list) - list of grid coordinates as tuples
        shapes (list) - list of shapes, each entry is a collection of points
            defining the vertices of a polygon

    Returns:
    """
    # Create a grid to plot the outline on
    # need a binary image to simulate transits
    grid_df = pd.DataFrame(data=grid, columns=['x', 'y'])  # df is easier to work with
    grid_shape = [grid_df.x.max(), grid_df.y.max()]
    outline = np.zeros(grid_shape) 

    for shape in shapes:
        points_df = pd.DataFrame(data=shape, columns=['x', 'y'])
        polygon = Polygon(points_df)
        test_inside_mask = polygon.contains_points(grid)
        outline[grid_df[test_inside_mask].x, grid_df[test_inside_mask].y] = 1

    return outline

In [3]:
def generatePoints(n,r,xoffset = 0,yoffset = 0,rotation = 0,proportion = 1): ##Generates points of a regular polygon with n edges, radius of r*size
    ##Size is grid size, 0 < r < 0.5, rotation rotates the shape from having a point vertically down
    points = []
    ystretch = np.sqrt(proportion)
    xstretch = 1/np.sqrt(proportion)
    for i in range(n):
        y = int(ystretch*r*np.cos(i*2*np.pi/n + np.pi/n))
        x = int(xstretch*r*np.sin(i*2*np.pi/n + np.pi/n))
        nuy = int(x*np.sin(rotation)+y*np.cos(rotation) + yoffset)
        nux = int(x*np.cos(rotation)-y*np.sin(rotation) + xoffset)
        points.append([nuy,nux])
    return points

In [4]:
def getData(filename): ##Gets the time array from the TESS Light Curve
    fp = open("./"+filename+".pkl","rb") ##Loading the file
    data = pickle.load(fp)
    fp.close()
    
    q = data[11]==0
    times = data[6][q]
    flux = data[9][q]
    err = np.ones_like(flux)*np.nanstd(np.diff(flux))
    err /= np.sqrt(2)
    return times, flux, err

In [5]:
def extendLC(LC, overlapTimes, times):
    extendedLC = np.ones(len(times))##Start with full flux curve
    overlapIndex = 0
    for i in range(len(times)):##Iterate over the desired time array
        if times[i] == overlapTimes[overlapIndex]: ##When the desired time has caught up to the light curve from ebt, use the ebt values
            if overlapIndex < len(overlapTimes) - 2:##Only do this to the end of the ebt light curve
                overlapIndex += 1
                extendedLC[i]  = LC[overlapIndex]
        
    return extendedLC, times

In [83]:
def model(sides,radius,rotation,stretch,velocity, tRef, times):
    points = generatePoints(sides,radius,(1/np.sqrt(stretch))*radius+1,50,rotation,stretch)
    pointsList = [points]
    grid = []
    for i in range(100):
        for j in range(int(3*(1/np.sqrt(stretch))*radius + 4)):
            grid.append((i,j))
    planetGrid = fillShape(grid,pointsList)
    planet = TransitingImage(opacitymat = planetGrid, v = velocity, t_ref = tRef, t_arr = times)
    planetLC, overlapTimes = planet.gen_LC(t_arr = times)
    planetLC, time = extendLC(planetLC, overlapTimes, times)
    return planetLC

In [102]:
def logLikelihood(theta, times, flux, fluxErr):
    sides, radius, rotation, stretch, velocity, tRef = theta
    fluxPredicted = model(int(sides), radius,rotation,stretch,velocity,tRef,times)
    lnl = -np.sum(((flux - fluxPredicted)**2) /(2*fluxErr**2))/len(flux)
    return lnl

In [65]:
def logPrior(theta, times):
    sides, radius, rotation, stretch, velocity, tRef = theta
    lnPrior = 0
    if times[0] < tRef < times[-1]:
        lnPrior += np.log(1/(times[-1]-times[0]))
    else:
        return -np.inf
    if  0  <= rotation < 2*np.pi and 0.1 < stretch < 10:
        lnPrior += np.log(1/(2*np.pi)) + 1/2
    else:
        return -np.inf
    if 3 < sides < 24 and 0 < radius < 50 and 0 < velocity < 50:
        lnPrior += np.log(1/21) + np.log(1/50) + np.log(1/50)
    else:
        return -np.inf
    return lnPrior
        

In [69]:
def logProbability(theta, times, flux, fluxErr):
    lp = logPrior(theta, times)
    if not np.isfinite(lp):
        return -np.inf
    ll = logLikelihood(theta, times, flux, fluxErr)
    return lp + ll

In [ ]:
times, flux, fluxErr = getData("tesslc_13203100000000")
tRef = np.median(times)
pos = [12, 5, 0, 1, 1, tRef]
for i in range(3):
    pos.append(pos)

sampler = emcee.EnsembleSampler(8,6,logProbability, args=(times,flux,fluxErr))
sampler.run_mcmc(pos,1000,progress = True);

In [ ]:
samples = sampler.get_chain()

In [97]:
def hRatio(theta0,theta1,flux, times, fluxErr):
    lnProb0 = logProbability(theta0, times, flux, fluxErr)
    lnProb1 = logProbability(theta1,times,flux,fluxErr)
    ratio = np.exp(lnProb1 - lnProb0)
    return ratio
    

In [78]:
def proposeJump(theta, cov):
    sides, radius, rotation, stretch, velocity, tRef = theta
    newSides = int(np.random.normal(sides, cov[0]))
    newRadius = int(np.random.normal(sides, cov[1]))
    newRotation = np.random.normal(rotation, cov[2])
    newStretch = np.log(np.random.normal(np.exp(stretch),cov[3]))
    newVelocity = np.random.normal(velocity,cov[4])
    newTRef = np.random.normal(tRef,cov[5])
    newTheta = (newSides, newRadius, newRotation, newStretch, newVelocity, newTRef)
    return newTheta

In [104]:
def mhMcmc(theta0, cov, steps, times, flux, fluxErr):
    
    positions = np.zeros((steps+1,len(theta0)))
    lnProbPos = -np.inf*np.ones(steps+1)
    acceptanceRatio = np.zeros_like(lnProbPos)
    accepted = 0
    
    positions[0] = theta0
    lnProbPos [0] = logProbability(theta0, times, flux, fluxErr)
    
    for step in np.arange(1,steps+1):
        proposal = proposeJump(positions[step -1],cov)
        h = hRatio(positions[step-1],proposal,flux,times,fluxErr)
        print(h)
        r = np.random.uniform()
        
        if h > r:
            accepted += 1
            positions[step] = proposal
            lnProbPos[step] = logProbability(proposal, times, flux, fluxErr)
        else:
            positions[step] = positions[step-1]
            lnProbPos[step] = lnProbPos[step-1]
        
        acceptanceRatio[step] = float(accepted)/steps
        
    return (positions, lnProbPos, acceptanceRatio)

In [108]:
times, flux, fluxErr = getData("tesslc_13203100000000")
tRef = np.median(times)
theta0= [12, 5, np.pi, 1, 1, tRef]
cov = [0.1,0.1,0.1,0.1,0.1,np.std(times)/2]
positions, lnProbPos, acceptanceRatio = mhMcmc(theta0, cov, 500, times, flux, fluxErr)
print(acceptanceRatio)
print(positions[-1])

/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:10: DeprecationWarning: elementwise comparison failed; this will raise an error in the future.
  # Remove the CWD from sys.path while we load stuff.


0.26271785395137837
0.5998673766345824
0.8279916495206675
0.0
1.514888754409159
0.9576296561515928
0.6266917816500592
1.4878231120421954
0.8341400338687519
1.342899912652822
1.441450622537873
1.480664613369045
1.030074224407191
0.9485102427601505
1.265704856157009
1.0745617012105708
0.936703819464895
1.157313549079693
1.0769628379164107
0.0
0.0
1.0197445364107955
0.0
0.975243093702795
0.0
0.9400631188166927
0.0
1.0190475993516364
0.0
0.0
0.0
0.9713029553203296
0.9925961769256668
1.0085452018878474
0.0
1.0320829347418603
0.9928251747170427
0.0
0.0
0.0
0.0
0.0
1.0715236028023412
0.0
0.9177445009804074
0.0
0.9315769628754305
1.0414628988084549
0.9964573345546056
0.8928026482405235
0.0
0.0
0.8574403051548883
0.0
1.047581909896547
0.0
0.9788896900736593
0.9316316717826404
0.0
1.1480236584130803
0.0
0.9756832932664303
0.0
0.9026004096948197
1.1036746391478527
0.0
0.0
1.1172931411809839
0.9850878138623658
0.0
0.0
1.015559018382818
0.0
0.0
1.015882057336166
0.9829308150603685
1.016521574194457

IndexError: index 0 is out of bounds for axis 0 with size 0